# Workbook 05 - Graph Traversals(Javascript)

## Overview

A graph consists of vertices and edges. Vertices are stored as documents in vertex collections and edges stored as documents in edge collections. The collections used in a graph and their relations are specified with edge definitions.

## Pre-requisite

Lets Assume 
- you have already made a tenant account, and have a username and password
- you have installed the jsc8 drivers as explained in section 01
- you have generated an API Key as explained in section 01


In [ ]:
#/* run this once to install javascript kernal and jsc8 in google colab, then reload, and then skip this
!npm install jsc8
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global  # */

## Step 05-A Importing Libraries

In [15]:
// Variables - Queries
const global_url = "gdn.paas.macrometa.io";
const email = "email"; // <-- Email goes here
const password = "password"; // <-- password goes here
const geo_fabric = "_system";
const collection_people = "CDRpeople";
const collection_calls = "CDRcalls";
const collection_cellsites = "CDRcellsites";
const collection_graph = "CDRgraphdocs";
const read_people = "FOR person IN CDRpeople RETURN person";
const read_calls = "FOR call IN CDRcalls RETURN call";
const person = "Lou Feaveer";
// const person = "Peria King"
const graph_traversal1 = `FOR c IN CDRpeople FILTER c.full_name == \"${person}\" FOR v IN 1..1 INBOUND c CDRcalls RETURN v`;
const graph_traversal2 = `FOR c IN CDRpeople FILTER c.full_name == \"${person}\" FOR v IN 1..1 OUTBOUND c CDRcalls RETURN v`;


## Step 05-B - Connecting to GDN

In [16]:
const jsc8 = require("jsc8");

// Simple Way
const client = new jsc8({
  url: "https://gdn.paas.macrometa.io",
  token: "XXXX",
  fabricName: "_system",
});

// ----- OR -----
// const client = new jsc8({url: "https://gdn.paas.macrometa.io", apiKey: "XXXX", fabricName: '_system'});

// To use advanced options
// const client = new jsc8("https://gdn.paas.macrometa.io");
// client.login(email, password).then(result => console.log("Login successfully", result)).catch(err => console.error("Error while login", err.message))


## Step 05-C - Creating Collections

In [35]:
async function createCollection() {
  console.log("\n 2. CREATE_COLLECTION");

  try {
    console.log(
      `Creating the CREATE_PEOPLE_VERTEX_COLLECTION ${collection_people}...`
    );
    let exists_coll = await client.hasCollection(collection_people);
    if (!exists_coll) {
      await client.createCollection(collection_people);
    }

    console.log(
      `Creating the CREATE_CALLS_EDGE_COLLECTION ${collection_calls}...`
    );
    exists_coll = await client.hasCollection(collection_calls);
    if (!exists_coll) {
      await client.createCollection(collection_calls, {}, true);
    }
  } catch (e) {
    console.log("Collection creation did not succeed due to " + e.message);
  }
}

createCollection();


## Step 05-D - Inserting Data - "People"

In [18]:
let people_data = [
  {
    full_name: "Kiel Dummer",
    first_name: "Kiel",
    last_name: "Dummer",
    city: "Burbank",
    state: "CA",
    address: "40317 5th Crossing",
    calling_nbr: "757-697-9065",
    _key: "757-697-9065",
  },
  {
    full_name: "Pernell Winspare",
    first_name: "Pernell",
    last_name: "Winspare",
    city: "San Diego",
    state: "CA",
    address: "596 Packers Pass",
    calling_nbr: "718-208-8096",
    _key: "718-208-8096",
  },
  {
    full_name: "Ava Kermath",
    first_name: "Ava",
    last_name: "Kermath",
    city: "Berkeley",
    state: "CA",
    address: "2 Doe Crossing Junction",
    calling_nbr: "765-623-5328",
    _key: "765-623-5328",
  },
  {
    full_name: "Tremain McGrah",
    first_name: "Tremain",
    last_name: "McGrah",
    city: "Torrance",
    state: "CA",
    address: "079 Russell Street",
    calling_nbr: "859-783-3227",
    _key: "859-783-3227",
  },
  {
    full_name: "Vidovik Boddam",
    first_name: "Vidovik",
    last_name: "Boddam",
    city: "Los Angeles",
    state: "CA",
    address: "3 Brentwood Crossing",
    calling_nbr: "703-265-1313",
    _key: "703-265-1313",
  },
  {
    full_name: "Oralie Goward",
    first_name: "Oralie",
    last_name: "Goward",
    city: "Los Angeles",
    state: "CA",
    address: "922 Columbus Park",
    calling_nbr: "617-815-8610",
    _key: "617-815-8610",
  },
  {
    full_name: "Lou Feaveer",
    first_name: "Lou",
    last_name: "Feaveer",
    city: "San Jose",
    state: "CA",
    address: "55223 Hooker Crossing",
    calling_nbr: "716-463-8993",
    _key: "716-463-8993",
  },
  {
    full_name: "Peria King",
    first_name: "Peria",
    last_name: "King",
    city: "Stockton",
    state: "CA",
    address: "8 Troy Plaza",
    calling_nbr: "713-707-8699",
    _key: "713-707-8699",
  },
];

async function insertDocumentMany() {
  console.log(`\n 3a. INSERT_PEOPLE_DATA in region ${global_url}`);
  await client.insertDocumentMany(collection_people, people_data);
}

insertDocumentMany();


## Step 05-E - Inserting Data - "Calls Records"

In [19]:
let callsData = [
  {
    calling_nbr: "757-697-9065",
    called_nbr: "716-463-8993",
    _from: "CDRpeople/757-697-9065",
    _to: "CDRpeople/716-463-8993",
    call_date: "1/4/2020",
    call_time: "13:33",
    call_duration: 30,
    cell_site: 4044703906,
  },
  {
    calling_nbr: "716-463-8993",
    called_nbr: "713-707-8699",
    _from: "CDRpeople/716-463-8993",
    _to: "CDRpeople/713-707-8699",
    call_date: "1/28/2020",
    call_time: "3:02",
    call_duration: 18,
    cell_site: 2289973823,
  },
  {
    calling_nbr: "765-623-5328",
    called_nbr: "713-707-8699",
    _from: "CDRpeople/765-623-5328",
    _to: "CDRpeople/713-707-8699",
    call_date: "1/28/2020",
    call_time: "3:02",
    call_duration: 18,
    cell_site: 2289973823,
  },
];

async function insertDocumentMany() {
  console.log(`\n 3a. INSERT_CALL_RECORDS_DATA in region ${global_url}`);
  await client.insertDocumentMany(collection_calls, callsData);
}

insertDocumentMany();


## Step 05-F - Creating a Graph

In [33]:
//Step4: Create a graph

async function createGraph() {
  console.log(`\n 4. CREATE_GRAPH region ${global_url}`);

  await client.createGraph(collection_graph, {
    edgeDefinitions: [
      {
        collection: collection_calls,
        from: [collection_people],
        to: [collection_people],
      },
    ],
  });
  console.log("Graph created");
}

createGraph().catch((err) =>
  console.log("Error while creating graph :", err.message)
);


## Step 05-G - Execute a Traversal - "Find outbound calls"

In [23]:
async function readData() {
  console.log(`5a. GRAPH_TRAVERSAL: Find outbound calls TO: ${person}`);
  let result = await client.executeQuery(graph_traversal1);
  console.log(result);
}

readData().catch((err) =>
  console.log("Error while execute a traversal :", err.message)
);

## Step 05-H - Execute a Traversal - "Find inbound calls"

In [24]:
async function readData() {
  console.log(`5b. GRAPH_TRAVERSAL: Find inbound calls FROM: ${person}`);
  result = await client.executeQuery(graph_traversal2);
  console.log(result);
}

readData().catch((err) =>
  console.log("Error while execute a traversal :", err.message)
);

## Step 05-I - Delete Data and Collections

In [36]:
// Step6: Delete Data

async function deleteData() {
  console.log(" 6. DELETE_DATA");
  await client.deleteGraph(collection_graph, true);
  console.log("Collection and Graph deleted");
}

deleteData().catch((err) =>
  console.log("Error while deleting collection and graphs:", err.message)
);

## Section Completed!

TBC...